**Objetivo do Notebook**: 

Este notebook apresenta uma análise exploratória voltada para o desenvolvimento de modelos de machine learning, realizada sobre uma base de dados contendo o histórico acadêmico dos alunos.

**Objetivo da análise**:

O objetivo da análise é proporcionar um entendimento geral dos alunos, permitindo conhecer melhor o conjunto de dados e suas características.

Busca-se também a verificação da conformidade da base de dados, identificando a presença de valores faltantes, duplicados, outliers e outras discrepâncias.

Este projeto visa desenvolver um modelo de machine learning capaz de estimar a probabilidade de um aluno conseguir emprego a partir de seu histórico escolar. Dessa forma, a análise pretende identificar características que possam auxiliar na criação da pipeline de pré-processamento e no treinamento do modelo. Por exemplo, podem existir vieses nos dados: será que a quantidade de projetos, a participação em workshops e as atividades extracurriculares não carregam vieses que podem "confundir" nossos modelos?

Assim, buscamos compreender o conjunto de dados para que essa análise contribua para as etapas subsequentes à criação do modelo.



# Importando bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carregando dados

In [3]:
data = pd.read_csv('../data/train.csv')

# Conhecendo a base de dados

In [4]:
data.head()

,cgpa,internships,projects,workshops_certifications,aptitude_test_score,soft_skills_rating,extra_curricular_activities,placement_training,ssc_marks,hsc_marks,n_job_applications,personality_type,placement_status,student_id
0,8.40,2,2,0,74,4.40,Yes,No,79,85,4,2,0,4085
1,7.60,1,2,1,75,4.30,Yes,Yes,80,67,7,4,1,2992
2,7.60,1,2,1,61,3.20,No,No,61,57,7,2,0,8623
3,7.50,1,2,0,78,4.40,Yes,Yes,61,71,7,2,0,8811
4,8.45,2,3,2,82,4.89,Yes,Yes,75,93,23,3,1,10072


In [5]:
pessoas = data.shape[0]

atributos = data.shape[1]

print(f"Há {pessoas} pessoas com cada uma tendo {atributos} atributos")

Há 9200 pessoas com cada uma tendo 14 atributos


Aqui vale uma breve observação sobre os atributos e seus respectivos significados:

* **cgpa (Cumulative Grade Point Average):**  Média geral das notas alcançadas pelo estudante. 
 
      Obs: Talvez possa ser entendido como um coeficiente de rendimento do aluno. Aparentemente cada instituição pode ser sua maneira de calcular o gpa.
* **internships:**  Quantidade de estágios que o estudante realizou.
* **projects:** Quantidade de projetos desenvolvidos pelo estudante.
* **workshops_certifications:** Quantidade de cursos e certificações realizadas pelo estudante.
* **aptitude_test_score:** Nota em testes de aptidão, que avaliam raciocínio lógico e quantitativo. 
      
      Obs: Quanto maior melhor ?
* **soft_skills_rating:** Avaliação de habilidades interpessoais e comunicação. 

      Obs: Quanto maior melhor?
* **extra_curricular_activities:**  Indica o nível de participação do estudante em atividades extracurriculares.
* **placement_training:** Indica se o estudante participou do treinamento de colocação profissional oferecido pela faculdade.
* **ssc_marks (Secondary School Certificate Marks)** Notas do ensino secundário.
* **hsc_marks (Higher Secondary certificate Marks):** Notas do Ensino Médio.
* **n_job_applications:** Número de vagas que o estudante se candidatou.
* **personality_type:** Tipo de personalidade do estudante (variável categórica). 

      Obs: Apesar de ser representado como um número, essa variável é categórica. Não sabemos o que seria uma personalidade 1,2 3 ou 4. 
* **placement_status:** Esse é o nosso target, indica se o estudante conseguiu um emprego (1 = colocado, 0 = não colocado).
* **Student id:** Identificação única de cada estudante

# Verificando a conformidade do dataset

Há valores nulos ? 

R: Não

In [6]:
print(f"Quantidade absoluta de valores nulos: {data.isna().sum().sum()}")

Quantidade absoluta de valores nulos: 0


Há valores duplicados ?

R: Não

In [7]:
print(f"Quantidade absoluta de valores duplicados: {data.duplicated().sum()}")

Quantidade absoluta de valores duplicados: 0


O pandas fez a inferência correta dos tipos de cada coluna da base ?

R: Aparentemente sim. Como temos poucas colunas, conseguimos visualizar facilmente pelo método .info().

Entretanto, há possibilidade de melhorias. Um dos defeitos do pandas consiste em alocar memória além do necessário para as colunas. Por exemplo, todas as colunas do tipo inteiro não vão além do valor 4, mas mesmo assim estão com o tipo int64. 

O tipo int64 abrange valores de  -9223372036854775808 a 9223372036854775807.

Para o nosso dataset, um int8 seria mais do que o suficiente. 

O tipo int8 abrange valores de -128 a 127.


O mesmo é valido para os valores float64. Neste caso, um float16 seria suficiente.

Para as colunas onde temos strings, o object poderia ser substituido por categorical, ou simplesmente podemos transformar seus valores para numéricos e setar o tipo para int8.

Todas essas melhorias são possíveis, entretanto, o dataset do jeito que está ocupa na memória irrisórios 1.7MB. Não precisamos nos preocupar com esses tipos superestimados. Não sofreremos com lentidão e nem com falta de memória.

In [8]:
data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cgpa                         9200 non-null   float64
 1   internships                  9200 non-null   int64  
 2   projects                     9200 non-null   int64  
 3   workshops_certifications     9200 non-null   int64  
 4   aptitude_test_score          9200 non-null   int64  
 5   soft_skills_rating           9200 non-null   float64
 6   extra_curricular_activities  9200 non-null   object 
 7   placement_training           9200 non-null   object 
 8   ssc_marks                    9200 non-null   int64  
 9   hsc_marks                    9200 non-null   int64  
 10  n_job_applications           9200 non-null   int64  
 11  personality_type             9200 non-null   int64  
 12  placement_status             9200 non-null   int64  
 13  student_id        

Há outliers ?

R: Não.

Através do método .describe() podemos ter um boa noção dos intervalos de valores para cada coluna. Inicialemente não encontramos nenhum indicio da presença de outliers. As colunas mais prováveis de encontramos outliers são as do tipo float. Entretanto, isso é esperado, pois sempre há estudantes fora da curva, tanto para cima quanto para baixo.

In [26]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
cgpa,9200.0,7.697162,0.644766,5.89,7.40,7.7,8.20,9.43
internships,9200.0,1.052065,0.666472,0.00,1.00,1.0,1.00,2.00
projects,9200.0,2.026196,0.868747,0.00,1.00,2.0,3.00,3.00
workshops_certifications,9200.0,1.016304,0.904360,0.00,0.00,1.0,2.00,3.00
aptitude_test_score,9200.0,79.448478,8.188742,59.00,73.00,80.0,87.00,96.00
soft_skills_rating,9200.0,4.324291,0.413076,2.98,4.00,4.4,4.70,5.28
ssc_marks,9200.0,69.109565,10.446232,50.00,59.00,70.0,78.00,96.00
hsc_marks,9200.0,74.526630,8.956924,55.00,67.00,73.0,83.00,95.00
n_job_applications,9200.0,16.537391,8.087944,3.00,10.00,17.0,23.00,33.00
personality_type,9200.0,2.486413,1.001374,1.00,2.00,2.0,3.00,4.00
